<a href="https://colab.research.google.com/github/hatkiet/Project_4/blob/main/1_HeartAttack_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataframe Basics

#### Note: this notebook should be run in Google Colab

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,391 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,537 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,179 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HeartAttack").getOrCreate()

In [ ]:
#Download datasets from kaggle
!kaggle datasets download -d kamilpytlak/personal-key-indicators-of-heart-disease --force

#unzip
!unzip -jo personal-key-indicators-of-heart-disease.zip "2022/heart_2022_with_nans.csv" -d /content/

Dataset URL: https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease
License(s): CC0-1.0
 28% 6.00M/21.4M [00:00<00:00, 62.0MB/s]
100% 21.4M/21.4M [00:00<00:00, 143MB/s] 
Archive:  personal-key-indicators-of-heart-disease.zip
  inflating: /content/heart_2022_with_nans.csv  


In [ ]:
#Read in dataset from CSV file into spark
from pyspark import SparkFiles
df = spark.read.csv(SparkFiles.get("/content/heart_2022_with_nans.csv"), sep=",", header=True)

In [ ]:
# Show DataFrame
df.show()

+-------+------+-------------+------------------+----------------+--------------------+------------------+----------+------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+-----------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+--------------------+--------------------+---------+---------------------+---------------+--------------+-----------------+-----+---------------+----------+------------+-------------+--------------------+----------------+--------+
|  State|   Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|     LastCheckupTime|PhysicalActivities|SleepHours|RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|DifficultyConcentrating|DifficultyWalking|DifficultyDressingBathing|Diffi

In [ ]:
# Print our schema
df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- GeneralHealth: string (nullable = true)
 |-- PhysicalHealthDays: string (nullable = true)
 |-- MentalHealthDays: string (nullable = true)
 |-- LastCheckupTime: string (nullable = true)
 |-- PhysicalActivities: string (nullable = true)
 |-- SleepHours: string (nullable = true)
 |-- RemovedTeeth: string (nullable = true)
 |-- HadHeartAttack: string (nullable = true)
 |-- HadAngina: string (nullable = true)
 |-- HadStroke: string (nullable = true)
 |-- HadAsthma: string (nullable = true)
 |-- HadSkinCancer: string (nullable = true)
 |-- HadCOPD: string (nullable = true)
 |-- HadDepressiveDisorder: string (nullable = true)
 |-- HadKidneyDisease: string (nullable = true)
 |-- HadArthritis: string (nullable = true)
 |-- HadDiabetes: string (nullable = true)
 |-- DeafOrHardOfHearing: string (nullable = true)
 |-- BlindOrVisionDifficulty: string (nullable = true)
 |-- DifficultyConcentrating: string (nullable = t

In [ ]:
# Show the columns
df.columns

['State',
 'Sex',
 'GeneralHealth',
 'PhysicalHealthDays',
 'MentalHealthDays',
 'LastCheckupTime',
 'PhysicalActivities',
 'SleepHours',
 'RemovedTeeth',
 'HadHeartAttack',
 'HadAngina',
 'HadStroke',
 'HadAsthma',
 'HadSkinCancer',
 'HadCOPD',
 'HadDepressiveDisorder',
 'HadKidneyDisease',
 'HadArthritis',
 'HadDiabetes',
 'DeafOrHardOfHearing',
 'BlindOrVisionDifficulty',
 'DifficultyConcentrating',
 'DifficultyWalking',
 'DifficultyDressingBathing',
 'DifficultyErrands',
 'SmokerStatus',
 'ECigaretteUsage',
 'ChestScan',
 'RaceEthnicityCategory',
 'AgeCategory',
 'HeightInMeters',
 'WeightInKilograms',
 'BMI',
 'AlcoholDrinkers',
 'HIVTesting',
 'FluVaxLast12',
 'PneumoVaxEver',
 'TetanusLast10Tdap',
 'HighRiskLastYear',
 'CovidPos']

In [ ]:
# Describe our data
df.describe()

DataFrame[summary: string, State: string, Sex: string, GeneralHealth: string, PhysicalHealthDays: string, MentalHealthDays: string, LastCheckupTime: string, PhysicalActivities: string, SleepHours: string, RemovedTeeth: string, HadHeartAttack: string, HadAngina: string, HadStroke: string, HadAsthma: string, HadSkinCancer: string, HadCOPD: string, HadDepressiveDisorder: string, HadKidneyDisease: string, HadArthritis: string, HadDiabetes: string, DeafOrHardOfHearing: string, BlindOrVisionDifficulty: string, DifficultyConcentrating: string, DifficultyWalking: string, DifficultyDressingBathing: string, DifficultyErrands: string, SmokerStatus: string, ECigaretteUsage: string, ChestScan: string, RaceEthnicityCategory: string, AgeCategory: string, HeightInMeters: string, WeightInKilograms: string, BMI: string, AlcoholDrinkers: string, HIVTesting: string, FluVaxLast12: string, PneumoVaxEver: string, TetanusLast10Tdap: string, HighRiskLastYear: string, CovidPos: string]

In [ ]:
df.select(["PhysicalHealthDays", "MentalHealthDays", "SleepHours", "BMI"]).describe().show()

+-------+------------------+-----------------+-----------------+------------------+
|summary|PhysicalHealthDays| MentalHealthDays|       SleepHours|               BMI|
+-------+------------------+-----------------+-----------------+------------------+
|  count|            434205|           436065|           439679|            396326|
|   mean| 4.347918609873217|4.382649375666472|7.022982675997716|28.529841822135072|
| stddev| 8.688911998349093|8.387474653346173|1.502425012888308|  6.55488866716511|
|    min|               0.0|              0.0|              1.0|             12.02|
|    max|               9.0|              9.0|              9.0|             99.64|
+-------+------------------+-----------------+-----------------+------------------+



In [ ]:
columns_to_remove = [
    'LastCheckupTime', 'RemovedTeeth', 'HadCOPD',
    'HadArthritis', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
    'DifficultyConcentrating', 'DifficultyDressingBathing', 'DifficultyErrands',
    'HeightInMeters', 'WeightInKilograms', 'HIVTesting', 'FluVaxLast12',
    'PneumoVaxEver', 'TetanusLast10Tdap', 'HighRiskLastYear', 'CovidPos'
]

df = df.drop(*columns_to_remove)
df.show()

+-------+------+-------------+------------------+----------------+------------------+----------+--------------+---------+---------+---------+-------------+---------------------+----------------+-----------+-----------------+--------------------+--------------------+---------+---------------------+---------------+-----+---------------+
|  State|   Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|PhysicalActivities|SleepHours|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadDepressiveDisorder|HadKidneyDisease|HadDiabetes|DifficultyWalking|        SmokerStatus|     ECigaretteUsage|ChestScan|RaceEthnicityCategory|    AgeCategory|  BMI|AlcoholDrinkers|
+-------+------+-------------+------------------+----------------+------------------+----------+--------------+---------+---------+---------+-------------+---------------------+----------------+-----------+-----------------+--------------------+--------------------+---------+---------------------+---------------+-----+------

In [ ]:
df.columns

['State',
 'Sex',
 'GeneralHealth',
 'PhysicalHealthDays',
 'MentalHealthDays',
 'PhysicalActivities',
 'SleepHours',
 'HadHeartAttack',
 'HadAngina',
 'HadStroke',
 'HadAsthma',
 'HadSkinCancer',
 'HadDepressiveDisorder',
 'HadKidneyDisease',
 'HadDiabetes',
 'DifficultyWalking',
 'SmokerStatus',
 'ECigaretteUsage',
 'ChestScan',
 'RaceEthnicityCategory',
 'AgeCategory',
 'BMI',
 'AlcoholDrinkers']

In [ ]:
column_order = [
        # User Info
        'State', 'Sex', 'AgeCategory', 'RaceEthnicityCategory','BMI',
        # General health
        'GeneralHealth', 'PhysicalHealthDays', 'MentalHealthDays', 'PhysicalActivities', 'ChestScan',
        # risks
        'SleepHours', 'DifficultyWalking', 'SmokerStatus', 'ECigaretteUsage', 'AlcoholDrinkers',
        # Medication History
        'HadHeartAttack', 'HadAngina',  'HadStroke', 'HadAsthma',
        'HadSkinCancer', 'HadDepressiveDisorder', 'HadKidneyDisease','HadDiabetes',
]
df = df.select(*column_order)
df.show()

+-------+------+---------------+---------------------+-----+-------------+------------------+----------------+------------------+---------+----------+-----------------+--------------------+--------------------+---------------+--------------+---------+---------+---------+-------------+---------------------+----------------+-----------+
|  State|   Sex|    AgeCategory|RaceEthnicityCategory|  BMI|GeneralHealth|PhysicalHealthDays|MentalHealthDays|PhysicalActivities|ChestScan|SleepHours|DifficultyWalking|        SmokerStatus|     ECigaretteUsage|AlcoholDrinkers|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadDepressiveDisorder|HadKidneyDisease|HadDiabetes|
+-------+------+---------------+---------------------+-----+-------------+------------------+----------------+------------------+---------+----------+-----------------+--------------------+--------------------+---------------+--------------+---------+---------+---------+-------------+---------------------+----------------+--

In [ ]:
rename_mapping = {
    'AgeCategory': "Age",
    'RaceEthnicityCategory': "Race",
    'GeneralHealth': "GenHealth",
    'PhysicalHealthDays': "PhysicalHealth",
    'PhysicalActivities': "PhysicalActivity",
    'MentalHealthDays': "MentalHealth",
    'HadAngina': "Angina",
    'HadHeartAttack': "HeartAttack",
    'HadStroke': "Stroke",
    'HadAsthma': "Asthma",
    'HadSkinCancer': "SkinCancer",
    'HadDepressiveDisorder': "Depressed",
    'HadKidneyDisease': "KidneyDisease",
    'DifficultyWalking': "DiffWalking",
    'HadDiabetes': "Diabetes",
    'SmokerStatus': "Smoking",
    'AlcoholDrinkers': "Drinking"
}

for name, rename in rename_mapping.items():
    df = df.withColumnRenamed(name, rename)

df.show()

+-------+------+---------------+--------------------+-----+---------+--------------+------------+----------------+---------+----------+-----------+--------------------+--------------------+--------+-----------+------+------+------+----------+---------+-------------+--------+
|  State|   Sex|            Age|                Race|  BMI|GenHealth|PhysicalHealth|MentalHealth|PhysicalActivity|ChestScan|SleepHours|DiffWalking|             Smoking|     ECigaretteUsage|Drinking|HeartAttack|Angina|Stroke|Asthma|SkinCancer|Depressed|KidneyDisease|Diabetes|
+-------+------+---------------+--------------------+-----+---------+--------------+------------+----------------+---------+----------+-----------+--------------------+--------------------+--------+-----------+------+------+------+----------+---------+-------------+--------+
|Alabama|Female|Age 80 or older|White only, Non-H...| NULL|Very good|           0.0|         0.0|              No|       No|       8.0|         No|        Never smoked|Not 

In [ ]:
from pyspark.sql.functions import col

# Count NaN values for each column
nan_counts = df.select([col(c).isNull().alias(c) for c in df.columns]) \
               .groupBy() \
               .count() \
               .collect()[0]

# Display NaN counts
print("NaN Value Distribution:")
for col_name, nan_count in zip(df.columns, nan_counts):
    print(f"{col_name}: {nan_count}")

NaN Value Distribution:
State: 445132


In [ ]:
# Drop rows with NaN values
df1 = df.dropna()

# Show DataFrame after dropping NaN values
df1.show()

+-------+------+---------------+--------------------+-----+---------+--------------+------------+----------------+---------+----------+-----------+--------------------+--------------------+--------+-----------+------+------+------+----------+---------+-------------+--------+
|  State|   Sex|            Age|                Race|  BMI|GenHealth|PhysicalHealth|MentalHealth|PhysicalActivity|ChestScan|SleepHours|DiffWalking|             Smoking|     ECigaretteUsage|Drinking|HeartAttack|Angina|Stroke|Asthma|SkinCancer|Depressed|KidneyDisease|Diabetes|
+-------+------+---------------+--------------------+-----+---------+--------------+------------+----------------+---------+----------+-----------+--------------------+--------------------+--------+-----------+------+------+------+----------+---------+-------------+--------+
|Alabama|Female|Age 80 or older|White only, Non-H...|26.57|Excellent|           0.0|         0.0|              No|       No|       6.0|         No|        Never smoked|Neve

In [ ]:
df1.dtypes

[('State', 'string'),
 ('Sex', 'string'),
 ('Age', 'string'),
 ('Race', 'string'),
 ('BMI', 'string'),
 ('GenHealth', 'string'),
 ('PhysicalHealth', 'string'),
 ('MentalHealth', 'string'),
 ('PhysicalActivity', 'string'),
 ('ChestScan', 'string'),
 ('SleepHours', 'string'),
 ('DiffWalking', 'string'),
 ('Smoking', 'string'),
 ('ECigaretteUsage', 'string'),
 ('Drinking', 'string'),
 ('HeartAttack', 'string'),
 ('Angina', 'string'),
 ('Stroke', 'string'),
 ('Asthma', 'string'),
 ('SkinCancer', 'string'),
 ('Depressed', 'string'),
 ('KidneyDisease', 'string'),
 ('Diabetes', 'string')]

In [ ]:
# Create a temporary view of the Dataframe
df1.createOrReplaceTempView("heart_data")

In [ ]:
# SQL query to calculate percentage of 'Yes' and 'No' in 'HeartAttack' column
# Use Spark SQL query to count the occurrences of each value and then calculate the percentages.

heart_attack_percentages = spark.sql("""
    SELECT ROUND(COUNT(CASE WHEN HeartAttack = 'Yes' THEN 1 END) / COUNT(*) * 100, 2) AS yes_percentage,
           ROUND(COUNT(CASE WHEN HeartAttack = 'No' THEN 1 END) / COUNT(*) * 100, 2) AS no_percentage
    FROM heart_data
""")

# Show the result
heart_attack_percentages.show()

+--------------+-------------+
|yes_percentage|no_percentage|
+--------------+-------------+
|          5.31|        94.69|
+--------------+-------------+



# Converting PySpark DataFrame to Pandas DataFrame

In [ ]:
import pandas as pd
pandas_df = df1.toPandas()

In [ ]:
pandas_df

,State,Sex,Age,Race,BMI,GenHealth,PhysicalHealth,MentalHealth,PhysicalActivity,ChestScan,...,ECigaretteUsage,Drinking,HeartAttack,Angina,Stroke,Asthma,SkinCancer,Depressed,KidneyDisease,Diabetes
0,Alabama,Female,Age 80 or older,"White only, Non-Hispanic",26.57,Excellent,0.0,0.0,No,No,...,Never used e-cigarettes in my entire life,No,No,No,No,No,Yes,No,No,No
1,Alabama,Female,Age 55 to 59,"White only, Non-Hispanic",25.61,Very good,2.0,3.0,Yes,No,...,Never used e-cigarettes in my entire life,No,No,No,No,No,Yes,No,No,No
2,Alabama,Female,Age 40 to 44,"White only, Non-Hispanic",21.77,Fair,2.0,0.0,Yes,Yes,...,Never used e-cigarettes in my entire life,Yes,No,No,No,No,No,No,No,No
3,Alabama,Male,Age 80 or older,"White only, Non-Hispanic",26.08,Poor,1.0,0.0,No,No,...,Never used e-cigarettes in my entire life,No,Yes,No,Yes,No,No,No,No,Yes
4,Alabama,Female,Age 80 or older,"Black only, Non-Hispanic",22.96,Very good,0.0,0.0,Yes,No,...,Never used e-cigarettes in my entire life,Yes,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318929,Virgin Islands,Male,Age 65 to 69,"Multiracial, Non-Hispanic",29.86,Good,0.0,15.0,Yes,No,...,Never used e-cigarettes in my entire life,Yes,No,No,Yes,No,No,No,No,Yes
318930,Virgin Islands,Male,Age 30 to 34,"White only, Non-Hispanic",31.19,Good,0.0,0.0,Yes,No,...,Never used e-cigarettes in my entire life,Yes,No,No,No,No,No,No,No,No
318931,Virgin Islands,Female,Age 50 to 54,"Black only, Non-Hispanic",28.66,Excellent,2.0,2.0,Yes,No,...,Never used e-cigarettes in my entire life,No,No,No,No,No,No,No,No,No
318932,Virgin Islands,Male,Age 70 to 74,"Black only, Non-Hispanic",32.55,Very good,0.0,0.0,No,Yes,...,Never used e-cigarettes in my entire life,No,Yes,No,No,Yes,No,No,No,No


In [ ]:
# Save to CSV
pandas_df.to_csv("/content/heart_2022_Spark.csv", index=False)

### Save *heart_2022_Spark.csv* into the *Resources* folder